In [4]:
import os

# Java
os.environ['JAVA_HOME'] = '/usr/lib/jvm/zulu-8'

# Python del entorno
os.environ['PYSPARK_PYTHON'] = '/home/debian1/BigData_UPAO/bigdata_env/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/debian1/BigData_UPAO/bigdata_env/bin/python3'

# Archivos de configuración de Hadoop/YARN
os.environ['HADOOP_CONF_DIR'] = '/opt/hadoop-3.3.6/etc/hadoop'
os.environ['YARN_CONF_DIR'] = '/opt/hadoop-3.3.6/etc/hadoop'

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .appName("BetGol-ETL-ML")
        .master("yarn")
        .config("spark.driver.memory", "3g")
        .config("spark.executor.memory", "3g")
        .config("spark.executor.cores", "2")
        .config("spark.sql.shuffle.partitions", "150")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .getOrCreate()
)

print("==============================================")
print("SparkSession iniciada correctamente")
print("Versión:", spark.version)
print("Master:", spark.sparkContext.master)


25/11/23 01:15:37 WARN Utils: Your hostname, vbox resolves to a loopback address: 127.0.1.1; using 192.168.18.70 instead (on interface enp0s3)
25/11/23 01:15:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/23 01:15:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/23 01:15:43 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


SparkSession iniciada correctamente
Versión: 3.5.1
Master: yarn


In [6]:
import os
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit, coalesce, to_date, trim
from pyspark.sql.types import IntegerType, DoubleType

# Carpeta RAW en HDFS
ruta_hdfs = "hdfs:///user/johan/data/raw/"

# Política de fechas antiguas
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Listar archivos desde HDFS
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
path = spark._jvm.org.apache.hadoop.fs.Path(ruta_hdfs)

archivos_status = fs.listStatus(path)

archivos_csv = [
    f.getPath().toString()
    for f in archivos_status
    if f.getPath().getName().endswith(".csv")
]

archivos_csv.sort()

cols_base = [
    "Div", "Date", "HomeTeam", "AwayTeam",
    "FTHG", "FTAG", "FTR",
    "HS", "AS", "HST", "AST", "HC", "AC",
    "HF", "AF", "HY", "AY", "HR", "AR",
    "B365H", "B365D", "B365A"
]

print("======================================================")
print(f"Archivos detectados en HDFS/raw/: {len(archivos_csv)}")
for a in archivos_csv:
    print(" -", a)
print(f"Columnas base definidas: {len(cols_base)}")

Archivos detectados en HDFS/raw/: 85
 - hdfs://localhost:9000/user/johan/data/raw/D1_1.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_10.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_11.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_12.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_13.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_14.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_15.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_16.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_17.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_2.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_3.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_4.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_5.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_6.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_7.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_8.csv
 - hdfs://localhost:9000/user/johan/data/raw/D1_9.csv
 - hdfs://localhost:9000/user/johan/d

In [7]:
print("Iniciando Carga Inteligente desde HDFS...")

lista_dfs_limpios = []

for archivo in archivos_csv:
    try:
        # Leer archivo individualmente desde HDFS
        df_temp = (
            spark.read
            .option("header", "true")
            .option("inferSchema", "true")
            .csv(archivo)
        )
        
        cols_actuales = df_temp.columns
        exprs_select = []

        # si falta una → NULL
        for c in cols_base:
            if c in cols_actuales:
                exprs_select.append(col(f"`{c}`"))
            else:
                exprs_select.append(lit(None).alias(c))

        # Over > 2.5
        if "B365>2.5" in cols_actuales:
            exprs_select.append(col("`B365>2.5`").alias("B365>2.5"))
        elif "BbAv>2.5" in cols_actuales:
            exprs_select.append(col("`BbAv>2.5`").alias("B365>2.5"))
        else:
            exprs_select.append(lit(None).alias("B365>2.5"))

        # Under < 2.5
        if "B365<2.5" in cols_actuales:
            exprs_select.append(col("`B365<2.5`").alias("B365<2.5"))
        elif "BbAv<2.5" in cols_actuales:
            exprs_select.append(col("`BbAv<2.5`").alias("B365<2.5"))
        else:
            exprs_select.append(lit(None).alias("B365<2.5"))

        # Construir DF limpio final
        df_limpio = df_temp.select(exprs_select)
        lista_dfs_limpios.append(df_limpio)

    except Exception as e:
        print(f"Error leyendo {archivo}: {e}")

print(f"Completado. {len(lista_dfs_limpios)} DataFrames procesados.")

Iniciando Carga Inteligente desde HDFS...


Completado. 85 DataFrames procesados.


In [8]:
from pyspark.sql.functions import col
from functools import reduce
from pyspark.sql import DataFrame

print("Unificando datos desde 2010-2025...")

if len(lista_dfs_limpios) > 0:

    # Unión vertical respetando nombres de columnas
    df_f1_raw = reduce(DataFrame.unionByName, lista_dfs_limpios)

    print("=" * 70)
    print("CARGA EXITOSA DESDE HDFS")
    print(f"   - Total de partidos cargados: {df_f1_raw.count()}")
    print("=" * 70)

    print("\n--- Muestra de TODAS las columnas estandarizadas ---")

    # Se obtiene la lista de columnas en orden final
    cols_finales = df_f1_raw.columns

    # Mostrar primeras 5 filas protegiendo nombres con backticks
    df_f1_raw.select([col(f"`{c}`") for c in cols_finales]).show(5, truncate=False)

else:
    print("No existen DataFrames limpios para unir.")

Unificando datos desde 2010-2025...
CARGA EXITOSA DESDE HDFS


   - Total de partidos cargados: 29513

--- Muestra de TODAS las columnas estandarizadas ---


25/11/23 01:47:44 WARN DAGScheduler: Broadcasting large task binary with size 1450.1 KiB


+---+--------+----------+-------------+----+----+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+--------+--------+
|Div|Date    |HomeTeam  |AwayTeam     |FTHG|FTAG|FTR|HS |AS |HST|AST|HC |AC |HF |AF |HY |AY |HR |AR |B365H|B365D|B365A|B365>2.5|B365<2.5|
+---+--------+----------+-------------+----+----+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+--------+--------+
|D1 |07/08/09|Wolfsburg |Stuttgart    |2   |0   |H  |13 |14 |7  |4  |6  |3  |12 |12 |0  |0  |0  |0  |1.95 |3.5  |3.75 |1.59    |2.24    |
|D1 |08/08/09|Dortmund  |FC Koln      |1   |0   |H  |24 |7  |11 |0  |16 |1  |8  |10 |0  |1  |0  |0  |1.62 |3.75 |5.5  |1.73    |2.02    |
|D1 |08/08/09|Hertha    |Hannover     |1   |0   |H  |10 |15 |4  |3  |5  |3  |16 |20 |3  |2  |0  |0  |1.8  |3.5  |4.5  |1.8     |1.94    |
|D1 |08/08/09|Hoffenheim|Bayern Munich|1   |1   |D  |9  |9  |1  |3  |3  |10 |10 |28 |0  |2  |0  |0  |4.2  |3.4  |1.91 |1.65    |2.14    |
|D1 |08/08/09|Mainz     |Leverkuse

## Renombrado de columnas

In [9]:
mapeo_columnas = {
    # --- 1. Identificadores---
    "Div": "division",
    "Date": "fecha",
    "HomeTeam": "equipo_local",
    "AwayTeam": "equipo_visitante",

    # --- Target / Resultados---
    "FTHG": "goles_local",
    "FTAG": "goles_visitante",
    "FTR": "resultado_final",  # (H, D, A)

    # --- Estadísticas de Juego ---
    "HS": "tiros_totales_local",
    "AS": "tiros_totales_visitante",
    "HST": "tiros_a_puerta_local",
    "AST": "tiros_a_puerta_visitante",
    "HC": "corners_local",
    "AC": "corners_visitante",
    "HY": "tarjetas_amarillas_local",
    "AY": "tarjetas_amarillas_visitante",
    "HR": "tarjetas_rojas_local",
    "AR": "tarjetas_rojas_visitante",

    # Stats extra 
    "HF": "faltas_local",
    "AF": "faltas_visitante",

    # --- Cuotas de Negocio ---
    "B365H": "cuota_local",
    "B365D": "cuota_empate",
    "B365A": "cuota_visitante",
    "B365>2.5": "cuota_mas_2_5",
    "B365<2.5": "cuota_menos_2_5"
}

df_renombre = df_f1_raw

for col_antigua, col_nueva in mapeo_columnas.items():
    df_renombre = df_renombre.withColumnRenamed(col_antigua, col_nueva)

print("--- Nuevo Esquema Estandarizado ----")
df_renombre.printSchema()

print("\n--- Muestra de Datos ---")
df_renombre.show(5)

--- Nuevo Esquema Estandarizado ----
root
 |-- division: string (nullable = true)
 |-- fecha: string (nullable = true)
 |-- equipo_local: string (nullable = true)
 |-- equipo_visitante: string (nullable = true)
 |-- goles_local: integer (nullable = true)
 |-- goles_visitante: integer (nullable = true)
 |-- resultado_final: string (nullable = true)
 |-- tiros_totales_local: integer (nullable = true)
 |-- tiros_totales_visitante: integer (nullable = true)
 |-- tiros_a_puerta_local: integer (nullable = true)
 |-- tiros_a_puerta_visitante: integer (nullable = true)
 |-- corners_local: integer (nullable = true)
 |-- corners_visitante: integer (nullable = true)
 |-- faltas_local: integer (nullable = true)
 |-- faltas_visitante: integer (nullable = true)
 |-- tarjetas_amarillas_local: integer (nullable = true)
 |-- tarjetas_amarillas_visitante: integer (nullable = true)
 |-- tarjetas_rojas_local: integer (nullable = true)
 |-- tarjetas_rojas_visitante: integer (nullable = true)
 |-- cuota_loc

25/11/23 01:56:32 WARN DAGScheduler: Broadcasting large task binary with size 1450.4 KiB


## Calidad de datos

### nulos

In [10]:
from pyspark.sql.functions import col, count, when

print(f"Total de filas en F1 Renombrado: {df_renombre.count()}")
print("\n--- REPORTE DE NULOS ---")

df_renombre.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_renombre.columns
]).show(vertical=True)

Total de filas en F1 Renombrado: 29513

--- REPORTE DE NULOS ---


25/11/23 02:01:19 WARN DAGScheduler: Broadcasting large task binary with size 1274.9 KiB
[Stage 178:===================================================>   (79 + 2) / 85]

-RECORD 0---------------------------
 division                     | 8   
 fecha                        | 8   
 equipo_local                 | 8   
 equipo_visitante             | 8   
 goles_local                  | 8   
 goles_visitante              | 8   
 resultado_final              | 8   
 tiros_totales_local          | 11  
 tiros_totales_visitante      | 11  
 tiros_a_puerta_local         | 11  
 tiros_a_puerta_visitante     | 11  
 corners_local                | 11  
 corners_visitante            | 11  
 faltas_local                 | 13  
 faltas_visitante             | 13  
 tarjetas_amarillas_local     | 12  
 tarjetas_amarillas_visitante | 11  
 tarjetas_rojas_local         | 11  
 tarjetas_rojas_visitante     | 11  
 cuota_local                  | 17  
 cuota_empate                 | 17  
 cuota_visitante              | 17  
 cuota_mas_2_5                | 18  
 cuota_menos_2_5              | 18  



### Verificar nulos

In [11]:
from pyspark.sql.functions import col, count

print("--- AUDITORÍA DE DUPLICADOS ---")

cols_clave = ["fecha", "equipo_local", "equipo_visitante"]

df_duplicados = df_renombre \
    .groupBy(cols_clave) \
    .agg(count("*").alias("conteo")) \
    .filter(col("conteo") > 1)

# Contamos cuántos casos de duplicidad existen
num_duplicados = df_duplicados.count()
print(f"Partidos repetidos encontrados: {num_duplicados}")

if num_duplicados > 0:
    print("\nEjemplo de Partidos Repetidos")
    df_duplicados.show(5)

    fila = df_duplicados.first()
    print(f"Detalle para: {fila['equipo_local']} vs {fila['equipo_visitante']} el {fila['fecha']}")

    df_renombre.filter(
        (col("fecha") == fila["fecha"]) &
        (col("equipo_local") == fila["equipo_local"])
    ).show()

--- AUDITORÍA DE DUPLICADOS ---


Partidos repetidos encontrados: 1

Ejemplo de Partidos Repetidos


+-----+------------+----------------+------+
|fecha|equipo_local|equipo_visitante|conteo|
+-----+------------+----------------+------+
| NULL|        NULL|            NULL|     8|
+-----+------------+----------------+------+



Detalle para: None vs None el None
+--------+-----+------------+----------------+-----------+---------------+---------------+-------------------+-----------------------+--------------------+------------------------+-------------+-----------------+------------+----------------+------------------------+----------------------------+--------------------+------------------------+-----------+------------+---------------+-------------+---------------+
|division|fecha|equipo_local|equipo_visitante|goles_local|goles_visitante|resultado_final|tiros_totales_local|tiros_totales_visitante|tiros_a_puerta_local|tiros_a_puerta_visitante|corners_local|corners_visitante|faltas_local|faltas_visitante|tarjetas_amarillas_local|tarjetas_amarillas_visitante|tarjetas_rojas_local|tarjetas_rojas_visitante|cuota_local|cuota_empate|cuota_visitante|cuota_mas_2_5|cuota_menos_2_5|
+--------+-----+------------+----------------+-----------+---------------+---------------+-------------------+-----------------------+---

### verificar si todos los duplicados son iguales

In [12]:
from pyspark.sql.functions import col, count

print("--- AUDITORÍA COMPARATIVA DE DUPLICADOS ---")

# filas iguales
num_filas_total = df_renombre.count()
num_filas_unicas = df_renombre.distinct().count()
duplicados_exactos = num_filas_total - num_filas_unicas

print(f"Duplicados EXACTOS (Filas idénticas): {duplicados_exactos}")

#Mismo partido
cols_clave = ["fecha", "equipo_local", "equipo_visitante"]
duplicados_logicos = df_renombre.groupBy(cols_clave).count().filter(col("count") > 1).count()

print(f"Duplicados LÓGICOS (Mismo partido): {duplicados_logicos}")

if duplicados_logicos > duplicados_exactos:
    print("Hay filas que son el mismo partido pero tienen datos diferentes.")
    print("Usa la limpieza por Clave Lógica para no duplicar partidos en tu modelo.")
else:
    print("\nLos duplicados son copias exactas.")

--- AUDITORÍA COMPARATIVA DE DUPLICADOS ---


25/11/23 02:12:25 WARN DAGScheduler: Broadcasting large task binary with size 1673.6 KiB
25/11/23 02:12:31 WARN DAGScheduler: Broadcasting large task binary with size 1242.9 KiB
                                                                                

Duplicados EXACTOS (Filas idénticas): 7


[Stage 202:====================================================>  (81 + 2) / 85]

Duplicados LÓGICOS (Mismo partido): 1

Los duplicados son copias exactas.


### Verificar Integridad de columnas categoricas


In [13]:
from pyspark.sql.functions import col

print("--- INTEGRIDAD CATEGÓRICA ---")

print("\nLigas encontradas en 'division':")
df_renombre.select("division").distinct().show()

print("\nResultados encontrados en 'resultado_final' (Solo debe haber H, D, A):")
df_renombre.select("resultado_final").distinct().show()

--- INTEGRIDAD CATEGÓRICA ---

Ligas encontradas en 'division':


+--------+
|division|
+--------+
|      D1|
|      E0|
|      F1|
|      I1|
|     SP1|
|    NULL|
+--------+


Resultados encontrados en 'resultado_final' (Solo debe haber H, D, A):


[Stage 211:=================================================>     (77 + 2) / 85]

+---------------+
|resultado_final|
+---------------+
|              A|
|              H|
|              D|
|           NULL|
+---------------+



### Verificamos columnas numericas mayores a 0

In [14]:
print("--- VALIDACIÓN NUMÉRICA ---")

# Goles Negativos
errores_goles = df_renombre.filter(
    (col("goles_local").cast("int") < 0) |
    (col("goles_visitante").cast("int") < 0)
)
print(f"Partidos con goles negativos: {errores_goles.count()}")

errores_cuotas = df_renombre.filter(
    (col("cuota_local").isNotNull()) &
    (col("cuota_local").cast("double") <= 1.0) |

    (col("cuota_empate").isNotNull()) &
    (col("cuota_empate").cast("double") <= 1.0) |

    (col("cuota_visitante").isNotNull()) &
    (col("cuota_visitante").cast("double") <= 1.0)
)
print(f"Partidos con cuotas inválidas (<= 1.0): {errores_cuotas.count()}")

--- VALIDACIÓN NUMÉRICA ---


Partidos con goles negativos: 0


[Stage 217:================================================>      (75 + 2) / 85]

Partidos con cuotas inválidas (<= 1.0): 0


### Verificar incoherencias

In [15]:
from pyspark.sql.functions import col

print("--- COHERENCIA DEPORTIVA (Goles vs Resultado) ---")

incoherencia_local = (col("goles_local").cast("int") > col("goles_visitante").cast("int")) & (col("resultado_final") != "H")

incoherencia_visita = (col("goles_local").cast("int") < col("goles_visitante").cast("int")) & (col("resultado_final") != "A")

incoherencia_empate = (col("goles_local").cast("int") == col("goles_visitante").cast("int")) & (col("resultado_final") != "D")

df_incoherencias = df_renombre.filter(incoherencia_local | incoherencia_visita | incoherencia_empate)

conteo_errores = df_incoherencias.count()
print(f"Partidos con resultados contradictorios: {conteo_errores}")

if conteo_errores > 0:
    print("\nSe encontraron errores graves de lógica. Muestra:")
    df_incoherencias.select("fecha", "equipo_local", "equipo_visitante",
                            "goles_local", "goles_visitante", "resultado_final").show(5)
else:
    print("\nTodos los marcadores coinciden con su resultado (H/D/A).")

--- COHERENCIA DEPORTIVA (Goles vs Resultado) ---


[Stage 220:================================================>      (75 + 2) / 85]

Partidos con resultados contradictorios: 0

Todos los marcadores coinciden con su resultado (H/D/A).


# LIMPIEZA

# eliminar 8 filas vacias

In [17]:
from pyspark.sql.functions import col

print("--- ELIMINACIÓN DE REGISTROS COMPLETAMENTE VACÍOS ---")

# how='all' elimina filas iguales
df_f1_limpio = df_renombre.dropna(how='all')

# Verificación
filas_borradas = df_renombre.count() - df_f1_limpio.count()
print(f"Filas completamente vacías eliminadas: {filas_borradas}")
print(f"Filas restantes: {df_f1_limpio.count()}")

--- ELIMINACIÓN DE REGISTROS COMPLETAMENTE VACÍOS ---


25/11/23 02:29:22 WARN DAGScheduler: Broadcasting large task binary with size 1189.7 KiB
                                                                                

Filas completamente vacías eliminadas: 8


25/11/23 02:29:26 WARN DAGScheduler: Broadcasting large task binary with size 1189.7 KiB
[Stage 229:=================================================>     (77 + 2) / 85]

Filas restantes: 29505


## Eliminar cuotas

from pyspark.sql.functions import col

print("--- ELIMINACIÓN DE REGISTROS SIN CUOTAS ---")

col_criticas = ["cuota_local", "cuota_empate", "cuota_visitante", "cuota_mas_2_5", "cuota_menos_2_5"]

df_limpieza_2 = df_f1_limpio.dropna(subset=col_criticas)

# Verificación
filas_borradas = df_f1_limpio.count() - df_limpieza_2.count()

print(f"Filas sin cuotas eliminadas: {filas_borradas}")
print(f"Filas restantes y válidas: {df_limpieza_2.count()}")

print("\nPaso de eliminación de cuotas completado.")

## imputar faltantes

In [20]:
from pyspark.sql.functions import col

print("--- IMPUTACIÓN DE ESTADÍSTICAS CON CERO ---")

col_imputar = [
    "tiros_totales_local", "tiros_totales_visitante", "tiros_a_puerta_local",
    "tiros_a_puerta_visitante", "corners_local", "corners_visitante",
    "faltas_local", "faltas_visitante", "tarjetas_amarillas_local",
    "tarjetas_amarillas_visitante", "tarjetas_rojas_local", "tarjetas_rojas_visitante"
]

df_imputado = df_limpieza_2.fillna(0, subset=col_imputar)

print("Imputación con cero completada.")

--- IMPUTACIÓN DE ESTADÍSTICAS CON CERO ---
Imputación con cero completada.


## Verificar si no hay nulos

In [21]:
from pyspark.sql.functions import col, count, when

print(f"Total de filas limpias: {df_imputado.count()}")
print("\n--- REPORTE DE NULOS ---")

df_imputado.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_imputado.columns
]).show(vertical=True)

25/11/23 02:35:09 WARN DAGScheduler: Broadcasting large task binary with size 1261.0 KiB
                                                                                

Total de filas limpias: 29490

--- REPORTE DE NULOS ---


25/11/23 02:35:14 WARN DAGScheduler: Broadcasting large task binary with size 1847.1 KiB
[Stage 244:===================================================>   (79 + 2) / 85]

-RECORD 0---------------------------
 division                     | 0   
 fecha                        | 0   
 equipo_local                 | 0   
 equipo_visitante             | 0   
 goles_local                  | 0   
 goles_visitante              | 0   
 resultado_final              | 0   
 tiros_totales_local          | 0   
 tiros_totales_visitante      | 0   
 tiros_a_puerta_local         | 0   
 tiros_a_puerta_visitante     | 0   
 corners_local                | 0   
 corners_visitante            | 0   
 faltas_local                 | 0   
 faltas_visitante             | 0   
 tarjetas_amarillas_local     | 0   
 tarjetas_amarillas_visitante | 0   
 tarjetas_rojas_local         | 0   
 tarjetas_rojas_visitante     | 0   
 cuota_local                  | 0   
 cuota_empate                 | 0   
 cuota_visitante              | 0   
 cuota_mas_2_5                | 0   
 cuota_menos_2_5              | 0   



## Verificar duplicados

In [22]:
from pyspark.sql.functions import col, count

print("=== BUSCANDO PARTIDOS DUPLICADOS ===")

cols_clave = ["fecha", "equipo_local", "equipo_visitante"]

df_dup = (
    df_imputado
    .groupBy(cols_clave)
    .agg(count("*").alias("conteo"))
    .filter(col("conteo") > 1)
)

num_dup = df_dup.count()

print(f"Partidos duplicados encontrados: {num_dup}")

if num_dup > 0:
    print("\n--- EJEMPLOS DE PARTIDOS DUPLICADOS ---")
    df_dup.show(10, truncate=False)
else:
    print("No hay duplicados según la clave lógica de partido.")


=== BUSCANDO PARTIDOS DUPLICADOS ===


25/11/23 02:41:25 WARN DAGScheduler: Broadcasting large task binary with size 1665.3 KiB
25/11/23 02:41:31 WARN DAGScheduler: Broadcasting large task binary with size 1255.4 KiB


Partidos duplicados encontrados: 0
No hay duplicados según la clave lógica de partido.


# ESTANDARIZACIÓN

In [24]:
df_imputado.printSchema()

root
 |-- division: string (nullable = true)
 |-- fecha: string (nullable = true)
 |-- equipo_local: string (nullable = true)
 |-- equipo_visitante: string (nullable = true)
 |-- goles_local: integer (nullable = true)
 |-- goles_visitante: integer (nullable = true)
 |-- resultado_final: string (nullable = true)
 |-- tiros_totales_local: integer (nullable = true)
 |-- tiros_totales_visitante: integer (nullable = true)
 |-- tiros_a_puerta_local: integer (nullable = true)
 |-- tiros_a_puerta_visitante: integer (nullable = true)
 |-- corners_local: integer (nullable = true)
 |-- corners_visitante: integer (nullable = true)
 |-- faltas_local: integer (nullable = true)
 |-- faltas_visitante: integer (nullable = true)
 |-- tarjetas_amarillas_local: integer (nullable = true)
 |-- tarjetas_amarillas_visitante: integer (nullable = true)
 |-- tarjetas_rojas_local: integer (nullable = true)
 |-- tarjetas_rojas_visitante: integer (nullable = true)
 |-- cuota_local: double (nullable = true)
 |-- cuo

## Cambiar formato de los años que tienen YY a formato YYYY

- Busca una barra "/" al final del texto, seguida de exactamente 2 dígitos.

- Reemplaza "/09" por "/2009"

In [26]:
from pyspark.sql.functions import col, regexp_replace

print("--- CORRECCIÓN DEL AÑO ---")

df_fechas_arregladas = df_imputado.withColumn(
    "fecha",
    regexp_replace(col("fecha"), r"/(\d{2})$", r"/20$1")
)

print("Muestra de fechas corregidas (Texto):")
df_fechas_arregladas.select("fecha").show(5)

--- CORRECCIÓN DEL AÑO ---
Muestra de fechas corregidas (Texto):
+----------+
|     fecha|
+----------+
|07/08/2009|
|08/08/2009|
|08/08/2009|
|08/08/2009|
|08/08/2009|
+----------+
only showing top 5 rows



25/11/23 02:49:05 WARN DAGScheduler: Broadcasting large task binary with size 1458.3 KiB


In [27]:
# VERIFICAR SI TODAS SE CONVIRTIERON
df_fechas_arregladas.select("fecha").filter(col("fecha").isNull()).count()

25/11/23 02:50:13 WARN DAGScheduler: Broadcasting large task binary with size 1422.0 KiB
                                                                                

0

## Estandarizar fecha

In [28]:
from pyspark.sql.functions import to_date
from pyspark.sql.types import DateType

print("--- CONVERSIÓN A TIPO DATE ---")

df_final = df_fechas_arregladas.withColumn(
    "fecha",
    to_date(col("fecha"), "dd/MM/yyyy").cast(DateType())
)

print("Columna fecha convertida exitosamente.")
df_final.printSchema()
df_final.select("fecha").show(10)

--- CONVERSIÓN A TIPO DATE ---
Columna fecha convertida exitosamente.
root
 |-- division: string (nullable = true)
 |-- fecha: date (nullable = true)
 |-- equipo_local: string (nullable = true)
 |-- equipo_visitante: string (nullable = true)
 |-- goles_local: integer (nullable = true)
 |-- goles_visitante: integer (nullable = true)
 |-- resultado_final: string (nullable = true)
 |-- tiros_totales_local: integer (nullable = true)
 |-- tiros_totales_visitante: integer (nullable = true)
 |-- tiros_a_puerta_local: integer (nullable = true)
 |-- tiros_a_puerta_visitante: integer (nullable = true)
 |-- corners_local: integer (nullable = true)
 |-- corners_visitante: integer (nullable = true)
 |-- faltas_local: integer (nullable = true)
 |-- faltas_visitante: integer (nullable = true)
 |-- tarjetas_amarillas_local: integer (nullable = true)
 |-- tarjetas_amarillas_visitante: integer (nullable = true)
 |-- tarjetas_rojas_local: integer (nullable = true)
 |-- tarjetas_rojas_visitante: integer (

25/11/23 02:51:21 WARN DAGScheduler: Broadcasting large task binary with size 1534.5 KiB


+----------+
|     fecha|
+----------+
|2009-08-07|
|2009-08-08|
|2009-08-08|
|2009-08-08|
|2009-08-08|
|2009-08-08|
|2009-08-08|
|2009-08-09|
|2009-08-09|
|2009-08-15|
+----------+
only showing top 10 rows



## Estandarizar texto

In [29]:
from pyspark.sql.functions import col, lower, regexp_replace, trim

print("--- ESTANDARIZACIÓN TEXTO ---")

def quitar_tildes(columna):
    c = col(columna)
    c = regexp_replace(c, "á", "a")
    c = regexp_replace(c, "é", "e")
    c = regexp_replace(c, "í", "i")
    c = regexp_replace(c, "ó", "o")
    c = regexp_replace(c, "ú", "u")
    c = regexp_replace(c, "ñ", "n")
    c = regexp_replace(c, "[^a-zA-Z0-9 ]", "")
    return trim(lower(c)) # Todo a minúsculas y sin espacios extra

df_final = df_final \
    .withColumn("equipo_local", quitar_tildes("equipo_local")) \
    .withColumn("equipo_visitante", quitar_tildes("equipo_visitante"))

print("Nombres limpiados (sin tildes, minúsculas).")
df_final.select("equipo_local", "equipo_visitante").distinct().show(5, truncate=False)

--- ESTANDARIZACIÓN TEXTO ---
Nombres limpiados (sin tildes, minúsculas).


25/11/23 02:54:04 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
[Stage 258:=====================================================> (83 + 2) / 85]

+-------------+----------------+
|equipo_local |equipo_visitante|
+-------------+----------------+
|bayern munich|leverkusen      |
|ein frankfurt|hamburg         |
|hoffenheim   |mgladbach       |
|werder bremen|stuttgart       |
|werder bremen|nurnberg        |
+-------------+----------------+
only showing top 5 rows



25/11/23 02:54:10 WARN DAGScheduler: Broadcasting large task binary with size 1344.2 KiB
                                                                                

# GUARDAR LIMPIEZA EN PARQUET

In [30]:
df_final.write.mode("overwrite").parquet("hdfs:///user/johan/data/silver/")

25/11/23 03:06:22 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
                                                                                